# Benchmark Al adatom

In [1]:
cd(@__DIR__)

using Pkg
Pkg.activate(".")

using AtomsCalculators
using ASEconvert # use this PR:https://github.com/mfherbst/ASEconvert.jl/pull/17, Pkg.add(url="https://github.com/tjjarvinen/ASEconvert.jl.git", rev="atomscalculators")
using Unitful: Å, nm
using PythonCall
ENV["PYTHON"] = "/SNS/users/ccu/miniconda3/envs/analysis/bin/python"
# install the following packages in julia REPL
# using CondaPkg
# CondaPkg.add_pip("IPython")
# CondaPkg.add_pip("nglview")
using StaticArrays: SVector


using GLMakie
using Molly


  Activating new project at `~/Documents/ABCD_J/EAM`
┌ Warning: 
│ 
│ 
│ Interface changes in v0.8.30, for `PeriodicSystems` submodule of CellListMap.
│ 
│ From v0.8.30 on, the `PeriodicSystems` submodule is deprecated and will be removed in future versions.
│ (code that works in v0.8.XX series will work in v0.8.3X through compatibility functions).
│ 
│ The same functionality can be achieved by using directly the `CellListMap` module and the 
│ `ParticleSystem` data structure. 
│ 
│ To migrate to the new interface, replace the code:
│ 
│ using CellListMap.PeriodicSystems  => using CellListMap
│ system = PeriodicSystem(...)       => system = ParticleSystem(...)
│ 
│ The new `ParticleSystem` interface supports non-periodic systems, by
│ not setting the `unitcell` field in the system (or set `unitcell = nothing`).
│ 
│ Suppress this warning by setting: ENV["CELLLISTMAP_8.3_WARNING"] = "false"
│ 
└ @ CellListMap.PeriodicSystems nothing:nothing


## 1. Import ASE and other Python modules

In [2]:
# Import ASE and other Python modules as needed
ase = pyimport("ase")
ase_view = pyimport("ase.visualize")
ase_plot = pyimport("ase.visualize.plot")
plt = pyimport("matplotlib.pyplot")

# Import Al EAM potential
fname = "Al99.eam.alloy"
EAM = pyimport("ase.calculators.eam") # python ASE-EAM calculator
eam_cal = ASEconvert.ASEcalculator(EAM.EAM(potential=fname))  # EAM calculater, converted to AtomsBase format


ASEcalculator(<py ase.calculators.eam.EAM object at 0x7f03b42c6180>)

## 2. Build an aluminum surface with adsorbate

In [3]:
al_LatConst = 4.0495
atom_mass = 26.9815u"u"  # Atomic mass of aluminum in grams per mole

# Build an (001) Al surface  
atoms_ase = ase.build.fcc100("Al", size=(5,5,6), vacuum = al_LatConst*2)
# The basis vectors on x and y are along 1/2<110> directions
ase.build.add_adsorbate(atoms_ase, "Al", al_LatConst/2, position=(al_LatConst*(2.5*sqrt(1/2)/2),al_LatConst*(2.5*sqrt(1/2)/2)))
# ase.build.add_adsorbate(atoms_ase, "Al", al_LatConst/2, "bridge")

atoms_ase.translate([al_LatConst*(sqrt(1/2)/4),al_LatConst*(sqrt(1/2)/4),0])
atoms_ase.wrap()

atoms_ase_cell = atoms_ase.get_cell()
box_size = pyconvert(Array{Float64}, [atoms_ase_cell[x,x] for x in range(0,2)])*0.1*u"nm"

# cell properties
# AtomsCalculators.potential_energy(atoms_ab, eam_cal)
# AtomsCalculators.forces(atoms_ab, eam_cal)
atoms_ase.get_pbc() 
# atoms_ase.get_cell() 
# atoms_ase.get_positions()

Python: array([ True,  True, False])

In [4]:
# Preview
ase_view.view(atoms_ase, viewer="x3d")


Python: <IPython.core.display.HTML object>

In [5]:
# Build an Julia AtomsBase abstract 
atoms_ab = pyconvert(AbstractSystem, atoms_ase)

FlexibleSystem(Al₁₅₁, periodic = TTF):
    bounding_box      : [ 14.3189        0        0;
                                0  14.3189        0;
                                0        0   26.323]u"Å"
    adsorbate_info    : PyDict{Any, Any}("cell" => [2.8637824638055176 0.0; 0.0 2.8637824638055176], "sites" => PyDict{Any, Any}("ontop" => (0, 0), "hollow" => (0.5, 0.5), "bridge" => (0.5, 0)), "top layer atom index" => 125)

             .----------------------------------.  
            /|                                  |  
           / |                                  |  
          /  |                                  |  
         /   |                                  |  
        /    |                                  |  
       /     |                                  |  
      /      |                                  |  
     /       |                                  |  
    *        |                                  |  
    |        |                                  |  

## 3. Define customized interaction type in AtomsCalculators

### 3.1 Define interaction

In [6]:
# AtomsCalculators class containing calculator and system
struct EAMInteraction
    calculator::Any  # Holds the ASE EAM calculator reference
    atoms_ab::Any    # Holds atoms representation compatible with ASE
end

# EAMInteraction with the ASE EAM calculator and system representation
eam_interaction = EAMInteraction(eam_cal, atoms_ab)

EAMInteraction(ASEcalculator(<py ase.calculators.eam.EAM object at 0x7f03b42c6180>), FlexibleSystem(Al₁₅₁, periodic = TTF, bounding_box = [[14.318912319027588, 0.0, 0.0], [0.0, 14.318912319027588, 0.0], [0.0, 0.0, 26.322999999999997]]u"Å"))

### 3.2 Customized convert_ase function for evaluating potential and interactions using the ASE EAM interaction

In [7]:
# Customized convert_ase function converting Molly system to ASE format: handling with charges
using UnitfulAtomic
import PeriodicTable
const uVelocity = sqrt(u"eV" / u"u")
function convert_ase_custom(system::AbstractSystem{D}) where {D}
    # print("called by Molly")
    D != 3 && @warn "1D and 2D systems not yet fully supported."

    n_atoms = length(system)
    pbc     = map(isequal(Periodic()), boundary_conditions(system))
    numbers = atomic_number(system)
    masses  = ustrip.(u"u", atomic_mass(system))

    symbols_match = [
        PeriodicTable.elements[atnum].symbol == string(atomic_symbol(system, i))
        for (i, atnum) in enumerate(numbers)
    ]
    if !all(symbols_match)
        @warn("Mismatch between atomic numbers and atomic symbols, which is not " *
              "supported in ASE. Atomic numbers take preference.")
    end

    cell = zeros(3, 3)
    for (i, v) in enumerate(bounding_box(system))
        cell[i, 1:D] = ustrip.(u"Å", v)
    end

    positions = zeros(n_atoms, 3)
    for at = 1:n_atoms
        positions[at, 1:D] = ustrip.(u"Å", position(system, at))
    end

    velocities = nothing
    if !ismissing(velocity(system))
        velocities = zeros(n_atoms, 3)
        for at = 1:n_atoms
            velocities[at, 1:D] = ustrip.(uVelocity, velocity(system, at))
        end
    end

    # We don't map any extra atom properties, which are not available in ASE as this
    # only causes a mess: ASE could do something to the atoms, but not taking
    # care of the extra properties, thus rendering the extra properties invalid
    # without the user noticing.
    charges = nothing
    magmoms = nothing
    for key in atomkeys(system)
        if key in (:position, :velocity, :atomic_symbol, :atomic_number, :atomic_mass)
            continue  # Already dealt with
        elseif key == :charge
            charges = charge.(system.atoms) #### Using the charge() function in Molly!
        elseif key == :magnetic_moment
            magmoms = system[:, :magnetic_moment]
        else
            @warn "Skipping atomic property $key, which is not supported in ASE."
        end
    end

    # Map extra system properties
    info = Dict{String, Any}()
    for (k, v) in pairs(system)
        if k in (:bounding_box, :boundary_conditions)
            continue
        elseif k in (:charge, )
            info[string(k)] = ustrip(u"e_au", v)
        elseif v isa Quantity || (v isa AbstractArray && eltype(v) <: Quantity)
            # @warn("Unitful quantities are not yet supported in convert_ase. " *
            #       "Ignoring key $k")
        else
            info[string(k)] = v
        end
    end

    ase.Atoms(; positions, numbers, masses, magmoms, charges,
              cell, pbc, velocities, info)
end

convert_ase_custom (generic function with 1 method)

### 3.3 Force calculation

In [8]:
# Define customized AtomsCalculators here
function AtomsCalculators.potential_energy(system::Molly.System, interaction::EAMInteraction; kwargs...)
    # Convert Molly's system to ASE's Atoms format
    ase_atoms = convert_ase_custom(system)
    
    # Calculate potential energy using ASE's EAM calculator
    # energy = AtomsCalculators.potential_energy(ase_atoms, interaction.calculator)
    energy_py = interaction.calculator.ase_python_calculator.get_potential_energy(ase_atoms)
    energy = pyconvert(Float64, energy_py)*u"eV" # also consider unit conversion

    return energy
end

function AtomsCalculators.forces(system::Molly.System, interaction::EAMInteraction; kwargs...)
    # Convert Molly's system to ASE's Atoms format
    ase_atoms = convert_ase_custom(system)

    # Use ASE to calculate forces
    f = interaction.calculator.ase_python_calculator.get_forces(ase_atoms)

    # Reshape and rearrange into the jupyter SVector format
    tmp = pyconvert(Array{Float64}, f)
    vector_svector = [SVector{3}(tmp[i, j] for j in 1:3) for i in 1:size(tmp, 1)]
    FT = AtomsCalculators.promote_force_type(system, interaction.calculator.ase_python_calculator)
    tmp2 = [SVector{3}(tmp[i, j] for j in 1:3) for i in 1:size(tmp, 1)]
    tmp3 = reinterpret(FT, tmp2)

    return tmp3
end

## 4. Create Molly system

### 4.1 Convert atom positions to Molly's expected format (nanometers) and create Molly.Atom objects

In [9]:
# Get atom positions from previously defined ASE system
function get_positions(atoms_ase)
    positions = [(atom.position[1], atom.position[2], atom.position[3]) for atom in atoms_ase]
    return positions
end

# Convert each position from Ångströms to nanometers and ensure the conversion is applied element-wise.
atom_positions = [SVector(uconvert(nm, pos[1]), 
    uconvert(nm, pos[2]), uconvert(nm, pos[3])) for pos in get_positions(atoms_ab)]

# for LJ interactions only
# Assuming ϵ is specified in eV, convert it to kJ/mol
ϵ_kJ_per_mol = 0.1*u"eV"#* conversion_factor * u"kJ/mol"

molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                        #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                          ) for i in 1:length(atom_positions)]

151-element Vector{Atom{Int64, Quantity{Float64, 𝐌, Unitful.FreeUnits{(u,), 𝐌, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}, Quantity{Float64, 𝐋^2 𝐌 𝐍^-1 𝐓^-2, Unitful.FreeUnits{(kJ, mol^-1), 𝐋^2 𝐌 𝐍^-1 𝐓^-2, nothing}}}}:
 Atom with index 1, charge=0, mass=26.9815 u, σ=0.0 nm, ϵ=0.0 kJ mol^-1
 Atom with index 2, charge=0, mass=26.9815 u, σ=0.0 nm, ϵ=0.0 kJ mol^-1
 Atom with index 3, charge=0, mass=26.9815 u, σ=0.0 nm, ϵ=0.0 kJ mol^-1
 Atom with index 4, charge=0, mass=26.9815 u, σ=0.0 nm, ϵ=0.0 kJ mol^-1
 Atom with index 5, charge=0, mass=26.9815 u, σ=0.0 nm, ϵ=0.0 kJ mol^-1
 Atom with index 6, charge=0, mass=26.9815 u, σ=0.0 nm, ϵ=0.0 kJ mol^-1
 Atom with index 7, charge=0, mass=26.9815 u, σ=0.0 nm, ϵ=0.0 kJ mol^-1
 Atom with index 8, charge=0, mass=26.9815 u, σ=0.0 nm, ϵ=0.0 kJ mol^-1
 Atom with index 9, charge=0, mass=26.9815 u, σ=0.0 nm, ϵ=0.0 kJ mol^-1
 Atom with index 10, charge=0, mass=26.9815 u, σ=0.0 nm, ϵ=0.0 kJ mol^-1
 ⋮
 Atom with index 143, charge=

### 4.2 Create a Molly system

In [10]:
# Prepare velocities for Molly
# Assuming you've defined temperatures and want to initialize random velocities
temperatures = 0.0u"K"  # Example temperature
molly_velocities = [Molly.random_velocity(atom_mass, temperatures) for _ in molly_atoms]

# Specify boundary condition
boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])

# Create the Molly System with atoms, positions, velocities, and boundary
molly_system = Molly.System(
    atoms=molly_atoms,
    atoms_data = [AtomData(element="Al") for a in molly_atoms],
    coords=atom_positions,  # Ensure these are SVector with correct units
    velocities=molly_velocities,
    boundary=boundary_condition,
    general_inters=[eam_interaction],  # This needs to be filled with actual interaction objects compatible with Molly
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    loggers=(coords=CoordinateLogger(1),),
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/nm"  # Ensure these units are correctly specified
    )


System with 151 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}[1.4318912319027588 nm, 1.4318912319027588 nm, 2.6323 nm])

## 5. Energy minimization

In [11]:
# Define simulation parameters
timestep = 1.0u"fs"
num_steps = 100  # Number of simulation steps

# tol: the default value was 1000 kJ/mol/nm ~= 9.6e13 eV/m ~= 1.5e-5 J/M
simulator = SteepestDescentMinimizer(step_size=0.01u"nm", tol=1e-10u"kg*m*s^-2", log_stream=devnull)
# Run the simulation
Molly.simulate!(molly_system, simulator)



System with 151 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(nm,), 𝐋, nothing}}[1.4318912319027588 nm, 1.4318912319027588 nm, 2.6323 nm])

In [12]:
# the original simulation cell
print(AtomsCalculators.potential_energy(pyconvert(AbstractSystem, atoms_ase), eam_cal))
ase_view.view(atoms_ase, viewer="x3d")

-481.8178005493418 eV

Python: <IPython.core.display.HTML object>

In [13]:
# simulation cell after energy minimization
atoms_ase_sim = convert_ase_custom(molly_system)
print(AtomsCalculators.potential_energy(pyconvert(AbstractSystem, atoms_ase_sim), eam_cal))
ase_view.view(atoms_ase_sim, viewer="x3d")

-482.65719444328784 eV

Python: <IPython.core.display.HTML object>

## 6. Customized Simulator

### 6.1 Initialize

In [14]:
# Build an Julia AtomsBase abstract 
atoms_ab = pyconvert(AbstractSystem, atoms_ase)
# Convert each position from Ångströms to nanometers and ensure the conversion is applied element-wise.

atom_positions = [SVector(uconvert(nm, pos[1]), 
    uconvert(nm, pos[2]), uconvert(nm, pos[3])) for pos in get_positions(atoms_ab)]

molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                        #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                          ) for i in 1:length(atom_positions)]
# Create the Molly System with atoms, positions, velocities, and boundary
molly_system = Molly.System(
    atoms=molly_atoms,
    atoms_data = [AtomData(element="Al") for a in molly_atoms],
    coords=atom_positions,  # Ensure these are SVector with correct units
    velocities=molly_velocities,
    boundary=boundary_condition,
    general_inters=[eam_interaction],  # This needs to be filled with actual interaction objects compatible with Molly
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    loggers=(coords=CoordinateLogger(1),),
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/nm"  # Ensure these units are correctly specified
    )

atoms_ase_sim = convert_ase_custom(molly_system)
print(AtomsCalculators.potential_energy(pyconvert(AbstractSystem, atoms_ase_sim), eam_cal))
ase_view.view(atoms_ase_sim, viewer="x3d")

-481.8178005493418 eV

Python: <IPython.core.display.HTML object>

### 6.2 Mutation

In [15]:
# Define the RandomDisplacementsSimulator structure
struct RandomDisplacementsSimulator{D,L}
    step_size::D
    max_steps::Int
    log_stream::L
end

# Constructor for RandomDisplacementsSimulator
function RandomDisplacementsSimulator(;
                                       step_size=0.01, max_steps=100,
                                       log_stream=devnull)
    return RandomDisplacementsSimulator(step_size, max_steps, log_stream)
end

# Implement the simulate! function for RandomDisplacementsSimulator
function simulate!(sys, sim::RandomDisplacementsSimulator; n_threads::Integer=Threads.nthreads(), run_loggers=true)
    for step_n in 1:sim.max_steps
        # Move each particle in a random direction
        for i in 1:length(sys.coords)
            random_direction = randn(size(sys.coords[i]))
            # Normalize the direction and scale by step size
            # random_direction /= norm(random_direction)
            sys.coords[i] += sim.step_size * random_direction
        end
        
        # Optionally, here you could wrap coordinates, apply constraints, etc.
        # Wrap the coordinates to the boundary
        sys.coords = wrap_coords.(sys.coords, (sys.boundary,))
        neighbors = find_neighbors(sys, sys.neighbor_finder; n_threads=n_threads)
        E = potential_energy(sys, neighbors; n_threads=n_threads)

        # Log the step number (or other details as needed)
        # Run the loggers
        run_loggers!(sys, neighbors, step_n, run_loggers; n_threads=n_threads)
        println(sim.log_stream, "Step 0 - potential energy ",
                E, " - max force N/A - N/A")
    end
    return sys
end

simulate! (generic function with 1 method)

In [16]:
# Define simulation parameters
step_size = 0.01*u"nm"
max_steps = 100

simulator = RandomDisplacementsSimulator(step_size=step_size, max_steps=max_steps)
# Run the simulation
simulate!(molly_system, simulator)

# simulation cell after energy minimization
atoms_ase_sim = convert_ase_custom(molly_system)
print(AtomsCalculators.potential_energy(pyconvert(AbstractSystem, atoms_ase_sim), eam_cal))
ase_view.view(atoms_ase_sim, viewer="x3d")

62.74306414806517 eV

Python: <IPython.core.display.HTML object>

In [17]:
## visualize
using GLMakie
visualize(molly_system.loggers.coords, boundary_condition, "sim.mp4")

"sim.mp4"